## Simple llm

In [8]:
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate

# initialize the OpenAI LLM
llm = OpenAI(temperature=0.7)

prompt = PromptTemplate(
    input_variables=["topic"],
    template= "Suggest a catchy blog title about {topic}",
)

topic = 'AI Agents'

formatted_prompt = prompt.format(topic=topic)
# Call the LLM with the formatted prompt
blog_title = llm.predict(formatted_prompt)
print(f"Suggested blog title: {blog_title}")

Suggested blog title: 

"Unleashing the Power of AI Agents: Revolutionizing the Future"


### pdf reader

In [21]:
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.llms import OpenAI

# load the document
loader = TextLoader('agents.txt')
documents = loader.load()

# # print the document list
# for doc in documents:
#     print(doc.page_content)
#     print(len(doc.page_content))

# split the document into chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=80, chunk_overlap=20)
docs = text_splitter.split_documents(documents)

# convert the text into embeddings & store in FAISS
embeddings = OpenAIEmbeddings()

vectorstore = FAISS.from_documents(docs, embeddings)

retriever = vectorstore.as_retriever(search_kwargs={"k": 1})

# Manually retrive the document
query = "What is an agent?"
retrived_docs = retriever.get_relevant_documents(query)

# combine rerived text int a single prompt
retrived_text = "".join([doc.page_content for doc in retrived_docs])
# print the retrived text
# print(retrived_text)

# Call the LLM with the retrived text
llm = OpenAI(temperature=0.7)

# Manually pass the retrieved text to the llm
prompt = PromptTemplate(
    input_variables=["retrived_text"],
    template= "Answer the question based on the context: {retrived_text}",
)
formatted_prompt = prompt.format(retrived_text=retrived_text)
answer = llm.predict(formatted_prompt)
print(answer)




AI agents are the latest form of artificial intelligence that is rapidly gaining popularity in various industries. These digital helpers are designed to assist and support humans in various tasks, making our lives easier and more efficient. They are programmed to learn from data, analyze information, and make decisions based on the available data. With their ability to continuously learn and adapt, AI agents have the potential to revolutionize the way we live and work in the future.

What is the potential impact of AI agents on our lives and work?

The potential impact of AI agents on our lives and work is significant. They have the ability to make our lives easier and more efficient by assisting and supporting us in various tasks. Their continuous learning and adaptability can also lead to the revolutionization of the way we live and work, potentially improving productivity and efficiency in various industries. However, there may also be concerns about the potential replacement of h

### LLM Chain

In [ ]:
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

# initialize the OpenAI LLM
llm = OpenAI(temperature=0.7)

prompt = PromptTemplate(
    input_variables=["topic"],
    template= "Suggest a catchy blog title about {topic}",
)

# chain a llm 
chain = LLMChain(llm=llm, prompt=prompt)
# Call the chain with the topic
topic = 'AI Agents'
result = chain.run(topic)
print(f"Suggested blog title: {result}")


Suggested blog title: 
"Unleashing the Power of AI Agents: Revolutionizing the Future"


### RetrieveQA Chain

In [26]:
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.llms import OpenAI
from langchain.chains import RetrievalQA

# load the document
loader = TextLoader('agents.txt')
documents = loader.load()

# # print the document list
# for doc in documents:
#     print(doc.page_content)
#     print(len(doc.page_content))

# split the document into chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=80, chunk_overlap=20)
docs = text_splitter.split_documents(documents)

# convert the text into embeddings & store in FAISS
embeddings = OpenAIEmbeddings()

vectorstore = FAISS.from_documents(docs, embeddings)

retriever = vectorstore.as_retriever(search_kwargs={"k": 1})


# Call the LLM with the retrived text
llm = OpenAI(temperature=0.7)


qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,
)

query = "What is an agent?"
answer = qa_chain.run(query)
print(answer)


# # Manually pass the retrieved text to the llm
# prompt = PromptTemplate(
#     input_variables=["retrived_text"],
#     template= "Answer the question based on the context: {retrived_text}",
# )
# formatted_prompt = prompt.format(retrived_text=retrived_text)
# answer = llm.predict(formatted_prompt)
# print(answer)



An agent is a computer program or algorithm that is designed to perform specific tasks or make decisions on behalf of a user. It is typically used in artificial intelligence systems to assist with various tasks and automate processes.


### RunableSequene

In [3]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from dotenv import load_dotenv
from langchain_core.runnables import RunnableSequence

load_dotenv()

# # load the "OPENAI_API_KEY" key
# import os
# test = os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")
# print(test)


prompt1 = PromptTemplate(
    input_variables=["topic"],
    template= "Swhat a joke about {topic}",
)

prompt2 = PromptTemplate(
    input_variables=["joke"],
    template= "explain the following joke {joke}",
)

model = ChatOpenAI()

parser = StrOutputParser()

# create a chain
chain = RunnableSequence(
    prompt1,
    model,
    parser,
    prompt2,
    model,
    parser,
)

chain.invoke({"topic": "AI Agents"})

'This joke plays on the idea that AI agents and chatbots are both forms of artificial intelligence, but the AI agent in the joke wants to be with someone who is even more advanced in terms of artificial intelligence. So, it breaks up with the chatbot to pursue a more intelligent partner.'

### Runable Parallel

In [6]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from dotenv import load_dotenv
from langchain_core.runnables import RunnableSequence, RunnableParallel

load_dotenv()

prompt1 = PromptTemplate(
    template= "Generate a tweet about {topic}",
    input_variables=["topic"],
)

prompt2 = PromptTemplate(
    template= "Generate a LinkedIn post about {topic}",
    input_variables=["topic"],
)


model = ChatOpenAI()

parser = StrOutputParser()

parallel_chain = RunnableParallel(
    {
        "tweet": RunnableSequence(
            prompt1,
            model,
            parser,
        ),
        "LinkedIn post": RunnableSequence(
            prompt2,
            model,
            parser,
        ),
    }
)

# run the parallel chain
result = parallel_chain.invoke({"topic": "AI Agents"})
print(result)


{'tweet': 'AI agents are revolutionizing the way we interact with technology. From chatbots to autonomous cars, these intelligent systems are changing the game. #AI #technology #innovation', 'LinkedIn post': 'Excited to see the advancements in AI agents, revolutionizing the way businesses operate. These intelligent virtual assistants are streamlining processes, improving customer experiences, and boosting productivity. The future is here with AI agents leading the way! #AI #ArtificialIntelligence #TechTrends #Innovation'}


### Runable Passthrough

In [9]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from dotenv import load_dotenv
from langchain_core.runnables import RunnableSequence, RunnableParallel, RunnablePassthrough

load_dotenv()

prompt1 = PromptTemplate(
    input_variables=["topic"],
    template= "Swhat a joke about {topic}",
)

prompt2 = PromptTemplate(
    input_variables=["joke"],
    template= "explain the following joke {joke}",
)

model = ChatOpenAI()

parser = StrOutputParser()

# joke gen chain 
joke_gen_chain = RunnableSequence(
    prompt1,
    model,
    parser,
)

parallel_chain = RunnableParallel(
    {
        "joke": RunnablePassthrough(),
        "explanation": RunnableSequence(
            prompt2,
            model,
            parser,
        ),
    }
)

final_chain = RunnableSequence(
    joke_gen_chain,
    parallel_chain,
)

result = final_chain.invoke({"topic": "AI Agents"})
print(result)

{'joke': "Why did the AI Agent break up with its data? Because it couldn't handle the constant emotional feedback loops!", 'explanation': 'This joke plays on the idea that AI agents are usually programmed to process data and make decisions based on logic, rather than emotions. The punchline suggests that the AI agent broke up with its data (meaning it stopped processing the data) because it couldn\'t handle the "emotional feedback loops" that were occurring. This is humorous because it is absurd to think of an AI agent experiencing emotions and breaking up with its data.'}


### Runable Lambda

In [14]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from dotenv import load_dotenv
from langchain_core.runnables import RunnableSequence, RunnableParallel, RunnablePassthrough, RunnableLambda

load_dotenv()

def word_count(text: str) -> int:
    return len(text.split())

prompt1 = PromptTemplate(
    input_variables=["topic"],
    template= "Swhat a joke about {topic}",
)

model = ChatOpenAI()

parser = StrOutputParser()

joke_gen_chain = RunnableSequence(
    prompt1,
    model,
    parser,
)

parallel_chain = RunnableParallel(
    {
        "joke": RunnablePassthrough(),
        "Word_count": RunnableLambda(word_count)
    }
)

final_chain = RunnableSequence(
    joke_gen_chain,
    parallel_chain,
)
result = final_chain.invoke({"topic": "AI Agents"})
print(result)

 

{'joke': 'Why did the AI agent go broke? Because it kept spending all its money on microtransactions!', 'Word_count': 16}


### Runable Branch

In [15]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from dotenv import load_dotenv
from langchain_core.runnables import RunnableSequence, RunnableParallel, RunnablePassthrough, RunnableLambda, RunnableBranch

load_dotenv()

prompt1 = PromptTemplate(
    template= "Write a detaled report on the following topic: {topic}",
    input_variables=["topic"],
)

prompt2 = PromptTemplate(
    template= "Summarize the following report: {report}",
    input_variables=["report"],
)

model = ChatOpenAI()

parser = StrOutputParser()

report_gen_chain = RunnableSequence(
    prompt1,
    model,
    parser,
)

branch_chain = RunnableBranch(
    (lambda x : len(x.split()) > 500, RunnableSequence(
        prompt2,
        model,
        parser,
    )),
    RunnablePassthrough()
)

final_chain = RunnableSequence(
    report_gen_chain,
    branch_chain,
)

response = final_chain.invoke({"topic": "AI Agents"})
print(response)

Artificial Intelligence (AI) agents are intelligent systems that are capable of perceiving their environment, making decisions, and taking actions to achieve specific goals. These agents use a combination of algorithms and data to learn from their interactions with the environment and improve their performance over time. AI agents have a wide range of applications, from autonomous vehicles and virtual assistants to chatbots and recommendation systems.

One of the key features of AI agents is their ability to adapt to changing conditions in their environment. For example, an AI agent controlling a self-driving car needs to continuously monitor its surroundings, make decisions based on the information it receives, and adjust its actions accordingly. This adaptability is achieved through machine learning algorithms, which allow the agent to learn from experience and improve its performance over time.

Another important aspect of AI agents is their ability to interact with humans in a natu

### LCEL

In [16]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from dotenv import load_dotenv
from langchain_core.runnables import RunnableSequence, RunnableParallel, RunnablePassthrough, RunnableLambda, RunnableBranch

load_dotenv()

prompt1 = PromptTemplate(
    template= "Write a detaled report on the following topic: {topic}",
    input_variables=["topic"],
)

prompt2 = PromptTemplate(
    template= "Summarize the following report: {report}",
    input_variables=["report"],
)

model = ChatOpenAI()

parser = StrOutputParser()

report_gen_chain = prompt1 | model | parser 

branch_chain = RunnableBranch(
    (lambda x : len(x.split()) > 500, RunnableSequence(
        prompt2,
        model,
        parser,
    )),
    RunnablePassthrough()
)

final_chain = RunnableSequence(
    report_gen_chain,
    branch_chain,
)

response = final_chain.invoke({"topic": "AI Agents"})
print(response)

Introduction: AI agents, also known as artificial intelligence agents, are intelligent software systems that can perform tasks autonomously without human intervention. These agents are designed to learn from their environment, make decisions, and interact with other agents to achieve a specific goal. In this report, we will discuss the different types of AI agents, their applications, advantages, and challenges.

Types of AI Agents:
1. Reactive Agents: These agents are designed to react to their environment based on pre-defined rules or algorithms. They do not have the ability to learn from past experiences.
2. Deliberative Agents: These agents are capable of planning and reasoning about their actions based on their goals and objectives. They make decisions by considering long-term consequences.
3. Hybrid Agents: These agents combine reactive and deliberative approaches to make decisions in complex environments. They can learn from past experiences and adjust their behavior accordingly